In [ ]:
import cuvis_ai
import cuvis
import yaml
from UNet import FreshTwin2DUNet
import torch
import numpy as np
import cv2 as cv
from pathlib import Path

In [ ]:
def remove_prefix(s, prefix):
    if s.startswith(prefix):
        return s[len(prefix):]
    return s

In [ ]:
CONF = 'UNet_train_config.yaml'
WEIGHTS = 'strawberry_swir_unet.ckpt'
proc = None

## Load Model

First we create a cuvis.ai Graph using some the config parameters from our training.

In [ ]:
with open(CONF, 'r') as f:
    config = yaml.safe_load(f)

graph = cuvis_ai.pipeline.Graph('Strawberry_graph')

modelNode = cuvis_ai.node.wrap.make_node(FreshTwin2DUNet)(
    criterion=torch.nn.NLLLoss, num_classes=config["num_classes"], in_channels=config["pca_channels"])

## Create Pipeline

In this case our Graph only consists of one Node. The PCA preprocessing is included in our Model and already loaded from the checkpoint.

In [ ]:
graph = graph >> modelNode

## Load checkpoint

We can easily load the model we previously trained using `train.py` into our modelNode

In [ ]:
checkpoint = torch.load(WEIGHTS, weights_only=False)

state_dict = {remove_prefix(
    k, 'model.'): v for k, v in checkpoint['state_dict'].items()}

modelNode.model.load_state_dict(state_dict)
modelNode.initialized = True
modelNode.model.pca = checkpoint['pca']

## Save cuvis.ai graph

When saving the graph to disk, the whole pipeline and model code will be saved as well.

In [ ]:
graph.save_to_file('Strawberry_Unet.ai')

## Load cuvis.ai graph from file

We can now load the zip file again into any project without having to worry about the model class being available there.

In [ ]:
loaded = cuvis_ai.pipeline.Graph.load_from_file('Strawberry_Unet.ai.zip')

## Infer one cube

Using the forward function of our graph we can now infer any cubert SessionFile that has the same dimensions as the ones we trained our model with.

In [ ]:
MESU = '../data/Strawberry_SWIR/Strawberry_133_5_22_04_25.cu3s'

In [ ]:
sess = cuvis.SessionFile(str(MESU))
input_mesu = sess.get_measurement(0)

if "cube" not in input_mesu.data:
    if proc is None:
        # create processing context only once if there are session files without cubes
        proc = cuvis.ProcessingContext(sess)
        if Path(config["white_path"]).exists() and Path(config["dark_path"]).exists():
            proc.set_reference(cuvis.SessionFile(config["white_path"]).get_measurement(0), cuvis.ReferenceType.White)
            proc.set_reference(cuvis.SessionFile(config["dark_path"]).get_measurement(0), cuvis.ReferenceType.Dark)
            proc.processing_mode = cuvis.ProcessingMode.Reflectance
    mesu = proc.apply(input_mesu)
input_data = input_mesu.data["cube"].array
output = graph.forward(np.expand_dims(input_data.astype(np.float32),0)) 

## Display the result

In [ ]:
cv.imshow("input RGB", cv.cvtColor(cv.normalize(input_data[:,:,config["cube_rgb_channels"]], None, 0, 255, cv.NORM_MINMAX, 0),cv.COLOR_BGR2RGB))
output_img = np.zeros_like(output)
prediction = np.argmax(output, axis=2)
output_img[:, :, 0] = (prediction == 1).astype(np.float32) # Strawberry will be labeled red
output_img[:, :, 2] = (prediction == 2).astype(np.float32) # Bruise will be labled blue
 
cv.imshow("output", cv.cvtColor(cv.normalize(output_img, None, 255, 0, cv.NORM_MINMAX, 0),cv.COLOR_BGR2RGB))

cv.waitKey(-1)
cv.destroyAllWindows()